In [24]:
from matplotlib import pyplot as plt
import cv2
import numpy as np

# Descripción

Este notebook se utiliza para convertir las imágenes tal y como se describe en el paper "On the segmentation of plantar foot thermal images with Deep Learning".

Las imágenes se dividen por pie y los pies derecho son "flipeados" para estar en la misma dirección que los izquierdo. También se normaliza las imágenes de IR para que estén almacenadas en 8 bits.

In [2]:
dataset_dir = "/home/abian/Workspace/Thesis/Segmentation/DataTest/"
images_dir = os.path.join(dataset_dir, 'Images')
mask_dir = os.path.join(dataset_dir, 'Masks_Average_STAPLE')

In [3]:
import re

def get_IR_images(filenames: list) -> list:
    regex_pattern = "[^\s]+(IR_T[0-9]+)(\.(png))$"
    r = re.compile(regex_pattern)
    return list(filter(r.match, filenames))

def get_mask_images(subject: str, mask_dir: str) -> list:
    regex_pattern = "({})(.*)([L|R]_T[0-9]+.png)".format(subject)
    r = re.compile(regex_pattern)
    filenames = os.listdir(mask_dir)
    return list(filter(r.match, filenames))


In [96]:
saving_dir = "processed/images/"
saving_mask_dir = "processed/masks/"
subjects = sorted(os.listdir(images_dir))

for subject in subjects:
    subject_img_dir = os.path.join(images_dir, subject)
    ir_img_list = get_IR_images(os.listdir(subject_img_dir))
    subject_masks = get_mask_images(subject, mask_dir)
    
    subject_save_dir = os.path.join(saving_dir, subject)
    subject_mask_save_dir = os.path.join(saving_mask_dir, subject)
    if not os.path.exists(subject_save_dir):
        os.mkdir(subject_save_dir)
    if not os.path.exists(subject_mask_save_dir):
        os.mkdir(subject_mask_save_dir)
    
    for ir_img in sorted(ir_img_list):
        T = ir_img.split("_")[-1]
        left_mask_filename, right_mask_filename = sorted([s for s in subject_masks if T in s])
        
        left_mask = cv2.imread(os.path.join(mask_dir, left_mask_filename), cv2.IMREAD_ANYDEPTH)
        right_mask = cv2.imread(os.path.join(mask_dir, right_mask_filename), cv2.IMREAD_ANYDEPTH)
        crop_w_right = np.argwhere(right_mask>0)[:,1].max() + 10
        crop_left_mask = left_mask[0:288, crop_w_right:384]
        crop_right_mask = right_mask[0:288, 0:crop_w_right]
        
        #flip el pie derecho
        crop_right_mask = cv2.flip(crop_right_mask, 1)

        cv2.imwrite(os.path.join(subject_mask_save_dir, right_mask_filename), crop_right_mask)
        cv2.imwrite(os.path.join(subject_mask_save_dir, left_mask_filename), crop_left_mask)
     
        img = cv2.imread(os.path.join(subject_img_dir, ir_img), cv2.IMREAD_ANYDEPTH)
        crop_img_right = img[0:288, 0:crop_w_right]
        crop_img_left = img[0:288, crop_w_right:384]
        crop_img_right = (crop_img_right / np.max(crop_img_right))*255 # Normalizar para que pillow lo lea bien (8 bits)
        crop_img_left = (crop_img_left / np.max(crop_img_left))*255 # Normalizar para que pillow lo lea bien (8 bits)

        #flip el pie derecho
        crop_img_right = cv2.flip(crop_img_right, 1)

        cv2.imwrite(os.path.join(subject_save_dir, ir_img.split(".")[0]+"_R.png"), crop_img_right.astype(np.uint8))
        cv2.imwrite(os.path.join(subject_save_dir, ir_img.split(".")[0]+"_L.png"), crop_img_left.astype(np.uint8))



print("Fin!")

Fin!
